In [220]:
# PART A: DOWNLOAD ALL OF ELECTIONLINE WEEKLY

# import requests
# from bs4 import BeautifulSoup
# import os

# YEARS = reversed(range(2011, 2024))

# # Define user-agent to simulate a web browser request
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
# }

# base_url = 'https://electionline.org'

# for year in YEARS:
#     dir_path = f"electionline-weekly/{year}"
#     os.makedirs(dir_path, exist_ok=True)
    
#     url = f"{base_url}/electionline-weekly/{year}"
    
#     # Send an HTTP GET request with headers
#     response = requests.get(url, headers=headers)

#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     weeks = soup.find('ul', class_='weeks').find_all('li')
#     weeks = [f"{base_url}{week.find('a')['href']}" for week in weeks]

#     for week in weeks:
#         response = requests.get(week, headers=headers)
#         with open(f"{week.split('electionline.org/')[-1]}.html", 'w') as f:
#             f.write(response.text)


In [235]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

pd.options.display.html.use_mathjax = False

YEARS = reversed(range(2011, 2024))

job_df = pd.DataFrame()

for year in YEARS:
    dir = f"electionline-weekly/{year}"
    weeks = os.listdir(dir)

    for week in weeks:
        date = week[:5]
        
        with open(os.path.join(dir, week)) as f:
            text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
    
        # Find all divs with the class 'article-wrapper'
        divs_with_class = soup.find_all('div', class_='article-wrapper')
    
        # Create an empty list to store the job postings as dictionaries
    
        for div in divs_with_class:
            h2_tags = div.find_all('h2', string=re.compile(r'^job', re.I))
            
            if h2_tags:
                for h2_tag in h2_tags:
                    # Find all p elements within the div containing the matched h2 tag
                    job_paragraphs = div.find_all('p')[1:]  # Skip the first paragraph
    
                    for paragraph in job_paragraphs:
                        # Extract job information from the paragraph
                        link = paragraph.find('a')
                        job_title = link.get_text() if link is not None else ""
                        employer_match = re.search(r'\/a>[^,]*(?:,|-)\s*([^-–—]*)', str(paragraph))
                        employer = employer_match.group(1) if employer_match else ""
                        salary_match = re.search(r'Salary[^:]*:\s(.*?)(?=\.\s)', paragraph.get_text())
                        salary = salary_match.group(1) if salary_match else ""
                        description = paragraph.get_text()
                        job_link = link['href'] if link is not None else ""
    
                        # Append job information to the list as a dictionary
                        new_row = pd.DataFrame({'Job Title': job_title,
                                                'Employer': employer,
                                                'Salary': salary,
                                                'Description': description,
                                                'Link': job_link,
                                                'Date': date,
                                                'Year': year}, index=[0])
                        job_df = pd.concat([job_df, new_row], ignore_index=True)

def display(df):
    # Replace dollar signs to disable MathJax
    return df.replace('\$', '\\$', regex=True)

display(job_df)

,Job Title,Employer,Salary,Description,Link,Date,Year
0,Assistant County Clerk-Recorder,"Nevada County, California","\$111,810 – \$136,500","Assistant County Clerk-Recorder, Nevada County...",http://www.governmentjobs.com/careers/mynevada...,03-09,2023
1,Assistant Director,"Butler County, Pennsylvania","\$45,129.18-\$63,180.85","Assistant Director, Butler County, Pennsylvani...",https://www.butlercountypa.gov/Jobs.aspx?Uniqu...,03-09,2023
2,Assistant Manager,"Palm Beach County, Florida",,"Assistant Manager-Poll Worker Department, Palm...",https://www.votepalmbeach.gov/About-Us/Assista...,03-09,2023
3,Chief Deputy Registrar of Voters,"San Bernardino, California","\$85,425.60 – \$118,684.80","Chief Deputy Registrar of Voters, San Bernardi...",https://www.governmentjobs.com/jobs/3899743-0/...,03-09,2023
4,Chief Information Officer,Illinois State Board of Elections,"\$7,885 – \$13,237 Monthly","Chief Information Officer, Illinois State Boar...",https://www.governmentjobs.com/careers/illinoi...,03-09,2023
...,...,...,...,...,...,...,...
7269,Program Analyst,"<strong>Dept. of Defense</strong>, Arlington, ...",,"Program Analyst, Dept. of Defense, Arlington, ...",http://www.usajobs.gov/GetJob/ViewDetails/3023...,11-17,2011
7270,Executive Director & General Counsel,"constitutional, and administrative law and rul...",,Executive Director & General Counsel – Connect...,http://das.ct.gov/HRDocs/JobsExams/EXECUTIVE%2...,11-17,2011
7271,mmoretti@electionline.org,,,electionlineWeekly publishes election administ...,mailto:mmoretti@electionline.org,10-20,2011
7272,Associate Counsel for Voting Rights Project,</strong>The Lawyers’ Committee for Civil Righ...,,"Associate Counsel for Voting Rights Project, T...",http://www.lawyerscommittee.org/about?id=0038,10-20,2011
